Data Collection and Preprocessing

In [1]:
# Import necessary libraries for data analysis and visualization
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

# Suppress warnings to keep the output clean
warnings.filterwarnings('ignore')
tickers = ['O', 'AGNC', 'QYLD', 'SPYD', 'SPYI', 'CSHI', 'NUSI', 'JEPI']

def fetch_and_analyze_outliers(tickers, start_date='2000-01-01', end_date='2023-10-01'):
    """
    Fetches historical stock data for the given tickers, calculates monthly returns,
    identifies outliers based on Z-scores, and returns a dictionary of outliers.

    Parameters:
    tickers (list): List of stock tickers to analyze.
    start_date (str): Start date for fetching data in 'YYYY-MM-DD' format.
    end_date (str): End date for fetching data in 'YYYY-MM-DD' format.

    Returns:
    dict: A dictionary where keys are tickers and values are DataFrames of outliers.
    """
    outlier_data = {}
    
    for ticker in tickers:
        # Fetch historical data using yfinance
        stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False)

        # Resample data to monthly frequency using the last trading day of each month
        monthly_data = stock_data.resample('M').last()

        # Calculate monthly returns
        monthly_data['Monthly Return'] = monthly_data['Adj Close'].pct_change()

        # Drop rows with NaN values to handle missing data
        monthly_data.dropna(inplace=True)

        # Reset index to have 'Date' as a column
        monthly_data.reset_index(inplace=True)

        # Extract month and year from the 'Date' column
        monthly_data['Month'] = monthly_data['Date'].dt.month
        monthly_data['Year'] = monthly_data['Date'].dt.year

        # Calculate Z-scores of monthly returns
        monthly_data['Return Z-Score'] = np.abs(stats.zscore(monthly_data['Monthly Return']))

        # Identify outliers where Z-score > 3
        outliers = monthly_data[monthly_data['Return Z-Score'] > 3]
        
        # Store outliers for each ticker
        outlier_data[ticker] = outliers[['Date', 'Monthly Return', 'Return Z-Score']]
    
    return outlier_data

# Example usage
outliers_dict = fetch_and_analyze_outliers(tickers)

# Display outliers for each ticker
for ticker, outliers in outliers_dict.items():
    print(f"Outliers in Monthly Returns for {ticker}:")
    display(outliers)



Outliers in Monthly Returns for O:


,Date,Monthly Return,Return Z-Score
241,2020-03-31,-0.308197,5.246154


Outliers in Monthly Returns for AGNC:


,Date,Monthly Return,Return Z-Score
12,2009-06-30,0.291128,3.701751
59,2013-05-31,-0.225458,3.125183
141,2020-03-31,-0.371460,5.054676
171,2022-09-30,-0.286106,3.926679


Outliers in Monthly Returns for QYLD:


...

,Date,Monthly Return,Return Z-Score


Outliers in Monthly Returns for NUSI:


,Date,Monthly Return,Return Z-Score
29,2022-06-30,-0.138796,3.373582


Outliers in Monthly Returns for JEPI:


,Date,Monthly Return,Return Z-Score


# Seasonality Analysis

In [2]:
def calculate_monthly_statistics(data_monthly):
    """
    Calculate monthly statistics for the given DataFrame of monthly returns.

    Parameters:
    data_monthly (DataFrame): DataFrame containing monthly returns with a 'Month' column.

    Returns:
    DataFrame: A DataFrame containing monthly statistics including mean return, standard deviation,
                count, median return, percentage of positive months, and maximum drawdown.
    """
    # Group data by month to calculate statistics
    monthly_stats = data_monthly.groupby('Month').agg({
        'Monthly Return': ['mean', 'std', 'count', 'median'],
    })

    # Calculate the percentage of positive months
    positive_months = data_monthly[data_monthly['Monthly Return'] > 0].groupby('Month').count()['Monthly Return']
    total_months = data_monthly.groupby('Month').count()['Monthly Return']
    monthly_stats['Pos%'] = (positive_months / total_months) * 100

    # Flatten MultiIndex columns
    monthly_stats.columns = ['Mean Return', 'Std Dev', 'Count', 'Median Return', 'Pos%']

    # Function to calculate maximum drawdown for each month
    def calculate_drawdown(returns):
        cumulative = (1 + returns).cumprod()
        peak = cumulative.cummax()
        drawdown = (cumulative - peak) / peak
        return drawdown.min()

    # Calculate maximum drawdown for each month
    monthly_drawdowns = data_monthly.groupby('Month')['Monthly Return'].apply(calculate_drawdown)
    monthly_stats['Max Drawdown'] = monthly_drawdowns.values

    return monthly_stats

def analyze_multiple_tickers(tickers, start_date='2000-01-01', end_date='2023-10-01'):
    """
    Fetches historical stock data for multiple tickers, calculates monthly returns,
    and computes monthly statistics for each ticker.

    Parameters:
    tickers (list): List of stock tickers to analyze.
    start_date (str): Start date for fetching data in 'YYYY-MM-DD' format.
    end_date (str): End date for fetching data in 'YYYY-MM-DD' format.

    Returns:
    dict: A dictionary where keys are tickers and values are DataFrames of monthly statistics.
    """
    monthly_stats_dict = {}
    
    for ticker in tickers:
        # Fetch historical data using yfinance
        stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False)

        # Resample data to monthly frequency using the last trading day of each month
        monthly_data = stock_data.resample('M').last()

        # Calculate monthly returns
        monthly_data['Monthly Return'] = monthly_data['Adj Close'].pct_change()

        # Drop rows with NaN values to handle missing data
        monthly_data.dropna(inplace=True)

        # Reset index to have 'Date' as a column
        monthly_data.reset_index(inplace=True)

        # Extract month and year from the 'Date' column
        monthly_data['Month'] = monthly_data['Date'].dt.month
        monthly_data['Year'] = monthly_data['Date'].dt.year

        # Calculate monthly statistics
        monthly_stats = calculate_monthly_statistics(monthly_data)

        # Store statistics for each ticker
        monthly_stats_dict[ticker] = monthly_stats

    return monthly_stats_dict


monthly_stats_dict = analyze_multiple_tickers(tickers)

# Display monthly statistics for each ticker
for ticker, stats in monthly_stats_dict.items():
    print(f"Monthly Statistics for {ticker}:")
    display(stats)


Monthly Statistics for O:


,Mean Return,Std Dev,Count,Median Return,Pos%,Max Drawdown
Month,,,,,,
1,0.027311,0.074705,23,0.045359,69.565217,-0.241472
2,-0.004622,0.048045,24,0.005981,54.166667,-0.194968
3,0.019256,0.085676,24,0.044662,66.666667,-0.308197
4,0.028446,0.077144,24,0.021894,62.500000,-0.200222
5,-0.009930,0.043322,24,-0.010009,41.666667,-0.325677
6,0.015980,0.057900,24,0.009248,66.666667,-0.169549
7,0.023821,0.048772,24,0.034864,70.833333,-0.101106
8,0.013945,0.064801,24,0.022589,70.833333,-0.182630
9,-0.008692,0.053051,24,0.009392,62.500000,-0.366734


Monthly Statistics for AGNC:


,Mean Return,Std Dev,Count,Median Return,Pos%,Max Drawdown
Month,,,,,,
1,0.024442,0.054108,15,0.007658,60.000000,-0.060529
2,-0.015130,0.075233,15,0.002846,53.333333,-0.238581
3,0.008410,0.112303,15,0.030051,86.666667,-0.371460
4,0.030384,0.073935,15,0.016169,60.000000,-0.156605
5,0.001498,0.079887,15,0.020505,66.666667,-0.225458
6,0.012535,0.103221,16,0.010317,62.500000,-0.194935
7,0.016907,0.053273,16,0.010619,56.250000,-0.053348
8,0.015570,0.069412,16,0.006700,56.250000,-0.135315
9,-0.005124,0.097044,16,-0.003580,43.750000,-0.332867


Monthly Statistics for QYLD:


...

,Mean Return,Std Dev,Count,Median Return,Pos%,Max Drawdown
Month,,,,,,
1,0.002201,NaN,1,0.002201,100.0,0.0
2,0.003726,NaN,1,0.003726,100.0,0.0
3,0.006823,NaN,1,0.006823,100.0,0.0
4,0.004432,NaN,1,0.004432,100.0,0.0
5,0.004812,NaN,1,0.004812,100.0,0.0
6,0.005320,NaN,1,0.005320,100.0,0.0
7,0.003820,NaN,1,0.003820,100.0,0.0
8,0.004552,NaN,1,0.004552,100.0,0.0
9,0.002533,0.002284,2,0.002533,100.0,0.0


Monthly Statistics for NUSI:


,Mean Return,Std Dev,Count,Median Return,Pos%,Max Drawdown
Month,,,,,,
1,-0.004048,0.047734,4,0.013898,75.000000,-0.074145
2,-0.016771,0.030644,4,-0.019830,25.000000,-0.039629
3,0.007568,0.024083,4,-0.000108,50.000000,-0.011035
4,0.017678,0.071677,4,0.022253,75.000000,-0.073461
5,0.011231,0.026631,4,0.006662,50.000000,-0.015891
6,0.002320,0.094393,4,0.044728,75.000000,-0.138796
7,0.023908,0.020206,4,0.014126,100.000000,0.000000
8,0.011501,0.021376,4,0.011896,75.000000,-0.014781
9,-0.047228,0.023170,4,-0.040023,NaN,-0.104677


Monthly Statistics for JEPI:


,Mean Return,Std Dev,Count,Median Return,Pos%,Max Drawdown
Month,,,,,,
1,-0.010932,0.026758,3,-0.015964,33.333333,-0.034816
2,-0.009324,0.014878,3,-0.012342,33.333333,-0.034526
3,0.040031,0.021046,3,0.033705,100.000000,0.000000
4,0.004072,0.035343,3,0.022989,66.666667,-0.036703
5,-0.000179,0.017470,3,-0.000723,33.333333,-0.018082
6,0.001406,0.029352,4,0.005481,50.000000,-0.036557
7,0.036180,0.018073,4,0.036556,100.000000,0.000000
8,0.002937,0.023798,4,0.008680,50.000000,-0.029419
9,-0.035788,0.023243,4,-0.036062,NaN,-0.129812


# DCA Strategy Backtesting

In [6]:
def calculate_dca_performance(data_monthly, initial_investment=1000):
    """
    Calculate the Dollar-Cost Averaging (DCA) performance metrics for a given monthly data.

    Parameters:
    data_monthly (DataFrame): DataFrame containing monthly data with 'Adj Close' prices.
    initial_investment (float): The fixed investment amount for each month.

    Returns:
    dict: A dictionary containing DCA performance metrics.
    """
    # Calculate the number of shares bought each month
    data_monthly['DCA Shares'] = initial_investment / data_monthly['Adj Close']

    # Calculate cumulative shares over time
    data_monthly['DCA Cumulative Shares'] = data_monthly['DCA Shares'].cumsum()

    # Calculate portfolio value over time
    data_monthly['DCA Portfolio Value'] = data_monthly['DCA Cumulative Shares'] * data_monthly['Adj Close']

    # Calculate total amount invested
    total_invested = initial_investment * len(data_monthly)

    # Calculate total return
    dca_total_return = data_monthly['DCA Portfolio Value'].iloc[-1] - total_invested

    # Calculate monthly portfolio returns
    data_monthly['DCA Portfolio Monthly Return'] = data_monthly['DCA Portfolio Value'].pct_change().fillna(0)

    # Calculate performance metrics
    dca_volatility = data_monthly['DCA Portfolio Monthly Return'].std() * np.sqrt(12)
    dca_drawdown = (data_monthly['DCA Portfolio Value'] / data_monthly['DCA Portfolio Value'].cummax() - 1).min()
    dca_sharpe_ratio = (data_monthly['DCA Portfolio Monthly Return'].mean() / data_monthly['DCA Portfolio Monthly Return'].std()) * np.sqrt(12)

    # Calculate Sortino Ratio
    downside_returns = data_monthly['DCA Portfolio Monthly Return'][data_monthly['DCA Portfolio Monthly Return'] < 0]
    dca_sortino_ratio = (data_monthly['DCA Portfolio Monthly Return'].mean() / downside_returns.std()) * np.sqrt(12) if downside_returns.std() != 0 else 0

    return {
        "Total Return": dca_total_return,
        "Annualized Volatility": dca_volatility,
        "Maximum Drawdown": dca_drawdown,
        "Sharpe Ratio": dca_sharpe_ratio,
        "Sortino Ratio": dca_sortino_ratio
    }

# Example usage for multiple tickers
def analyze_dca_for_tickers(tickers, initial_investment=1000, start_date='2000-01-01', end_date='2023-10-01'):
    """
    Analyze DCA performance for multiple tickers.

    Parameters:
    tickers (list): List of stock tickers to analyze.
    initial_investment (float): The fixed investment amount for each month.
    start_date (str): Start date for fetching data in 'YYYY-MM-DD' format.
    end_date (str): End date for fetching data in 'YYYY-MM-DD' format.

    Returns:
    dict: A dictionary where keys are tickers and values are DCA performance metrics.
    """
    dca_performance_dict = {}
    
    for ticker in tickers:
        # Fetch historical data using yfinance
        stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False)

        # Resample data to monthly frequency using the last trading day of each month
        monthly_data = stock_data.resample('M').last()

        # Calculate DCA performance metrics
        dca_performance = calculate_dca_performance(monthly_data, initial_investment)
        dca_performance_dict[ticker] = dca_performance

    return dca_performance_dict


dca_performance_dict = analyze_dca_for_tickers(tickers)

# Display DCA performance metrics for each ticker
for ticker, metrics in dca_performance_dict.items():
    print(f"DCA Strategy Performance Metrics for {ticker}:")
    print(f"Total Return: ${metrics['Total Return']:,.2f}")
    print(f"Annualized Volatility: {metrics['Annualized Volatility']:.2%}")
    print(f"Maximum Drawdown: {metrics['Maximum Drawdown']:.2%}")
    print(f"Sharpe Ratio: {metrics['Sharpe Ratio']:.2f}")
    print(f"Sortino Ratio: {metrics['Sortino Ratio']:.2f} \n")


DCA Strategy Performance Metrics for O:
Total Return: $869,063.10
Annualized Volatility: 33.11%
Maximum Drawdown: -37.77%
Sharpe Ratio: 1.04
Sortino Ratio: 2.29 

DCA Strategy Performance Metrics for AGNC:
Total Return: $81,653.60
Annualized Volatility: 39.77%
Maximum Drawdown: -44.76%
Sharpe Ratio: 1.09
Sortino Ratio: 1.84 

DCA Strategy Performance Metrics for QYLD:
Total Return: $39,913.84
Annualized Volatility: 39.57%
Maximum Drawdown: -17.29%
Sharpe Ratio: 1.47
Sortino Ratio: 7.16 

DCA Strategy Performance Metrics for SPYD:
Total Return: $18,466.40
Annualized Volatility: 45.58%
Maximum Drawdown: -32.83%
Sharpe Ratio: 1.50
Sortino Ratio: 3.80 

DCA Strategy Performance Metrics for SPYI:
Total Return: $661.24
Annualized Volatility: 85.73%
Maximum Drawdown: 0.00%
Sharpe Ratio: 3.23
Sortino Ratio: nan 

DCA Strategy Performance Metrics for CSHI:
Total Return: $408.28
Annualized Volatility: 88.51%
Maximum Drawdown: 0.00%
Sharpe Ratio: 3.11
Sortino Ratio: nan 

DCA Strategy Performance

# Kelly Fractional Seasonal Investing

In [7]:
def calculate_kelly_fraction(data_monthly):
    """
    Calculate the Kelly Fraction for each month based on historical data.

    Parameters:
    data_monthly (DataFrame): DataFrame containing monthly return data with 'Month' and 'Monthly Return' columns.

    Returns:
    DataFrame: The input DataFrame with an additional 'Kelly Fraction' column.
    """
    # Calculate average positive and negative returns for each month
    win_avg = data_monthly[data_monthly['Monthly Return'] > 0].groupby('Month')['Monthly Return'].mean()
    loss_avg = data_monthly[data_monthly['Monthly Return'] <= 0].groupby('Month')['Monthly Return'].mean()

    # Create a new DataFrame for Kelly fraction calculations
    monthly_stats = pd.DataFrame(index=range(1, 13))  # 12 months in a year

    # Map win/loss averages back to the month index
    monthly_stats['Win Average'] = win_avg
    monthly_stats['Loss Average'] = loss_avg

    # Replace NaN with zero to avoid division errors
    monthly_stats.fillna(0, inplace=True)

    # Calculate Win/Loss Ratio
    monthly_stats['Win/Loss Ratio'] = np.where(monthly_stats['Loss Average'] != 0, 
                                               monthly_stats['Win Average'] / np.abs(monthly_stats['Loss Average']),
                                               0)

    # Calculate the percentage of positive months (Pos %)
    positive_months = data_monthly[data_monthly['Monthly Return'] > 0].groupby('Month').count()['Monthly Return']
    total_months = data_monthly.groupby('Month').count()['Monthly Return']
    monthly_stats['Pos%'] = (positive_months / total_months).fillna(0)

    # Calculate Kelly Fraction
    monthly_stats['Kelly Fraction'] = (monthly_stats['Pos%'] - (1 - monthly_stats['Pos%']) / monthly_stats['Win/Loss Ratio'])

    # Handle cases where Kelly Fraction may produce NaN or negative results
    monthly_stats['Kelly Fraction'] = monthly_stats['Kelly Fraction'].clip(lower=0, upper=0.5)

    # Merge Kelly Fraction back to the original data
    data_monthly = pd.merge(data_monthly, monthly_stats['Kelly Fraction'], left_on='Month', right_index=True, how='left')

    return data_monthly


def calculate_kelly_strategy_performance(data_monthly, starting_capital=100000):
    """
    Calculate the Kelly Fractional Seasonal Investing strategy performance metrics for the given monthly data.

    Parameters:
    data_monthly (DataFrame): DataFrame containing monthly data with 'Adj Close' prices and 'Kelly Fraction'.
    starting_capital (float): Initial capital to start the investment.

    Returns:
    dict: A dictionary containing Kelly strategy performance metrics.
    """
    # Initialize variables for the Kelly strategy
    cash = starting_capital
    kelly_portfolio_values = []
    kelly_shares_owned = 0

    # Implement the Kelly strategy
    for _, row in data_monthly.iterrows():
        # Ensure Kelly Fraction is not NaN or zero
        if pd.isna(row['Kelly Fraction']) or row['Kelly Fraction'] == 0:
            kelly_portfolio_values.append(kelly_shares_owned * row['Adj Close'] + cash)
            continue

        # Determine investment amount based on Kelly Fraction
        invest_amount = cash * row['Kelly Fraction']
        # Calculate shares to buy
        shares_to_buy = invest_amount / row['Adj Close']
        # Update shares owned and cash
        kelly_shares_owned += shares_to_buy
        cash -= invest_amount
        # Calculate portfolio value
        portfolio_value = kelly_shares_owned * row['Adj Close'] + cash
        kelly_portfolio_values.append(portfolio_value)

    # Add portfolio values to the DataFrame
    data_monthly['Kelly Portfolio Value'] = kelly_portfolio_values

    # Calculate total return for Kelly strategy
    kelly_total_return = data_monthly['Kelly Portfolio Value'].iloc[-1] - starting_capital

    # Calculate monthly returns for Kelly strategy
    data_monthly['Kelly Portfolio Monthly Return'] = data_monthly['Kelly Portfolio Value'].pct_change().fillna(0)

    # Calculate performance metrics for Kelly strategy
    if len(data_monthly['Kelly Portfolio Monthly Return'].dropna()) > 1:
        kelly_volatility = data_monthly['Kelly Portfolio Monthly Return'].std() * np.sqrt(12)
        kelly_drawdown = (data_monthly['Kelly Portfolio Value'] / data_monthly['Kelly Portfolio Value'].cummax() - 1).min()
        kelly_sharpe_ratio = (data_monthly['Kelly Portfolio Monthly Return'].mean() / data_monthly['Kelly Portfolio Monthly Return'].std()) * np.sqrt(12)
        
        # Calculate Sortino Ratio
        downside_returns = data_monthly['Kelly Portfolio Monthly Return'][data_monthly['Kelly Portfolio Monthly Return'] < 0]
        downside_deviation = downside_returns.std() * np.sqrt(12) if len(downside_returns) > 0 else 0
        kelly_sortino_ratio = (data_monthly['Kelly Portfolio Monthly Return'].mean() / downside_deviation) if downside_deviation > 0 else 0
    else:
        # Handle cases with insufficient data
        kelly_volatility = 0
        kelly_drawdown = 0
        kelly_sharpe_ratio = 0
        kelly_sortino_ratio = 0

    return {
        "Total Return": kelly_total_return,
        "Annualized Volatility": kelly_volatility,
        "Maximum Drawdown": kelly_drawdown,
        "Sharpe Ratio": kelly_sharpe_ratio,
        "Sortino Ratio": kelly_sortino_ratio
    }


def analyze_kelly_for_tickers(tickers, start_date='2000-01-01', end_date='2023-10-01', starting_capital=100000):
    """
    Analyze Kelly Fractional Seasonal Investing strategy performance for multiple tickers.

    Parameters:
    tickers (list): List of stock tickers to analyze.
    start_date (str): Start date for fetching data in 'YYYY-MM-DD' format.
    end_date (str): End date for fetching data in 'YYYY-MM-DD' format.
    starting_capital (float): Initial capital to start the investment.

    Returns:
    dict: A dictionary where keys are tickers and values are Kelly strategy performance metrics.
    """
    kelly_performance_dict = {}
    
    for ticker in tickers:
        # Fetch historical data using yfinance
        stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False)

        # Resample data to monthly frequency using the last trading day of each month
        monthly_data = stock_data.resample('M').last()

        # Calculate monthly returns
        monthly_data['Monthly Return'] = monthly_data['Adj Close'].pct_change()

        # Drop rows with NaN values to handle missing data
        monthly_data.dropna(inplace=True)

        # Reset index to have 'Date' as a column
        monthly_data.reset_index(inplace=True)

        # Extract month and year from the 'Date' column
        monthly_data['Month'] = monthly_data['Date'].dt.month
        monthly_data['Year'] = monthly_data['Date'].dt.year

        # Calculate Kelly Fraction
        monthly_data = calculate_kelly_fraction(monthly_data)

        # Calculate Kelly strategy performance
        kelly_performance = calculate_kelly_strategy_performance(monthly_data, starting_capital)
        kelly_performance_dict[ticker] = kelly_performance

    return kelly_performance_dict


# Example usage
kelly_performance_dict = analyze_kelly_for_tickers(tickers)

# Display Kelly strategy performance metrics for each ticker
for ticker, metrics in kelly_performance_dict.items():
    print(f"Kelly Strategy Performance Metrics for {ticker}:")
    print(f"Total Return: ${metrics['Total Return']:,.2f}")
    print(f"Annualized Volatility: {metrics['Annualized Volatility']:.2%}")
    print(f"Maximum Drawdown: {metrics['Maximum Drawdown']:.2%}")
    print(f"Sharpe Ratio: {metrics['Sharpe Ratio']:.2f}")
    print(f"Sortino Ratio: {metrics['Sortino Ratio']:.2f} \n")


Kelly Strategy Performance Metrics for O:
Total Return: $1,601,381.93
Annualized Volatility: 20.89%
Maximum Drawdown: -38.02%
Sharpe Ratio: 0.68
Sortino Ratio: 0.08 

Kelly Strategy Performance Metrics for AGNC:
Total Return: $354,243.20
Annualized Volatility: 25.20%
Maximum Drawdown: -48.09%
Sharpe Ratio: 0.53
Sortino Ratio: 0.05 

Kelly Strategy Performance Metrics for QYLD:
Total Return: $74,436.42
Annualized Volatility: 11.64%
Maximum Drawdown: -22.73%
Sharpe Ratio: 0.55
Sortino Ratio: 0.06 

Kelly Strategy Performance Metrics for SPYD:
Total Return: $65,236.71
Annualized Volatility: 18.87%
Maximum Drawdown: -36.60%
Sharpe Ratio: 0.44
Sortino Ratio: 0.04 

Kelly Strategy Performance Metrics for SPYI:
Total Return: $0.00
Annualized Volatility: 0.00%
Maximum Drawdown: 0.00%
Sharpe Ratio: nan
Sortino Ratio: 0.00 

Kelly Strategy Performance Metrics for CSHI:
Total Return: $0.00
Annualized Volatility: 0.00%
Maximum Drawdown: 0.00%
Sharpe Ratio: nan
Sortino Ratio: 0.00 

Kelly Strategy 

# Comparative Analysis

**Comparative Analysis of DCA and Kelly Strategy**

In this comparison between the **Dollar-Cost Averaging (DCA)** strategy and the **Kelly strategy**, DCA consistently outperformed Kelly in terms of _risk-adjusted returns_, as seen through both the **Sharpe Ratio** and the **Sortino Ratio**. The DCA strategy had significantly better Sharpe and Sortino ratios across all assets, indicating that it delivered more favorable returns relative to risk and downside risk. For example, **O** had a DCA Sharpe Ratio of **1.04** and Sortino Ratio of **2.29**, while Kelly lagged far behind with a Sharpe Ratio of **0.68** and an extremely low Sortino Ratio of **0.08**. Similarly, for **AGNC**, DCA had a Sharpe Ratio of **1.09** and Sortino Ratio of **1.84**, compared to Kelly’s **0.53** and **0.05**, respectively. **JEPI** showed the most striking contrast, with DCA yielding a Sharpe Ratio of **2.11** and Sortino Ratio of **24.42**, while Kelly had a much lower Sharpe Ratio of **0.75** and Sortino Ratio of **0.12**. The only outlier was **SPYI** and **CSHI**, where both strategies underperformed or generated no returns.

Moreover, the Kelly strategy typically exhibited _lower volatility_, but it also came with _higher maximum drawdowns_ for most assets, such as **AGNC** and **SPYD**, where Kelly’s drawdowns were worse than DCA's, despite reducing volatility. Notably, **NUSI** had a much better Sortino Ratio under DCA (**8.82**) compared to Kelly’s minimal **0.01**, showing that Kelly failed to manage downside risk as effectively. Overall, DCA provided better total returns in assets like **NUSI**, **JEPI**, and **QYLD**, while controlling downside risk more efficiently, making it the more favorable strategy in this analysis.

However, it's important to address some data limitations, particularly small sample sizes for certain assets like **SPYI**, **CSHI**, and **NUSI**, which may skew the reliability of the statistics such as standard deviation, median return, and drawdown. Several assets had limited data for monthly analysis, leading to potential _NaN_ values and less confidence in the results. _Expanding the dataset_ or extending the time period could provide more robust insights, while also ensuring consistency in the calculation of key metrics such as _maximum drawdown_ and _volatility_.